In [0]:
WITH base AS (
  SELECT
      year,
      month,
      make_date(year,month,day) AS trip_date,

      SUM(total_amount) AS revenue,
      SUM(trip_distance) AS distance,
      COUNT(*) AS trips,
      SUM(passenger_count) AS passengers

  FROM bi_taxi
  GROUP BY year,month,make_date(year,month,day)
),

unpivoted AS (
  SELECT year,month,trip_date,'Revenue' AS metric_type,revenue AS metric_value FROM base
  UNION ALL
  SELECT year,month,trip_date,'Distance',distance FROM base
  UNION ALL
  SELECT year,month,trip_date,'Trips',trips FROM base
  UNION ALL
  SELECT year,month,trip_date,'Passengers',passengers FROM base
)

SELECT
    year,
    month,
    trip_date,
    metric_type,
    metric_value,

    (metric_value - MIN(metric_value) OVER (PARTITION BY metric_type))
    /
    NULLIF(
        MAX(metric_value) OVER (PARTITION BY metric_type)
        - MIN(metric_value) OVER (PARTITION BY metric_type),
        0
    ) AS heat_value

FROM unpivoted
ORDER BY trip_date;
